In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import os

# Boat Type Classification using CNN and MobileNetV2

This notebook walks through a deep learning project for classifying boat types from images. We will build, train, and evaluate two models: a custom Convolutional Neural Network (CNN) and a transfer learning model using MobileNetV2.

In [5]:
import os
import shutil
import random

# Step 0: Split Dataset into Train, Validation, and Test sets
base_dir = '../boat_type_classification_dataset'
output_dir = './data'

# Create directories for train, validation, and test sets
train_dir = os.path.join(output_dir, 'train')
validation_dir = os.path.join(output_dir, 'validation')
test_dir = os.path.join(output_dir, 'test')

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

os.makedirs(train_dir)
os.makedirs(validation_dir)
os.makedirs(test_dir)

# Get the list of classes (boat types)
classes = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

for cls in classes:
    # Create subdirectories for each class in train, validation, and test
    os.makedirs(os.path.join(train_dir, cls))
    os.makedirs(os.path.join(validation_dir, cls))
    os.makedirs(os.path.join(test_dir, cls))

    # Get all image file names for the class
    src_dir = os.path.join(base_dir, cls)
    all_files = os.listdir(src_dir)
    random.shuffle(all_files)

    # Define split ratios
    train_ratio = 0.7
    validation_ratio = 0.15
    test_ratio = 0.15

    # Calculate split indices
    train_split = int(len(all_files) * train_ratio)
    validation_split = int(len(all_files) * (train_ratio + validation_ratio))

    # Get file lists for each set
    train_files = all_files[:train_split]
    validation_files = all_files[train_split:validation_split]
    test_files = all_files[validation_split:]

    # Function to copy files
    def copy_files(files, dest_dir):
        for f in files:
            shutil.copy(os.path.join(src_dir, f), os.path.join(dest_dir, cls, f))

    # Copy files to their respective directories
    copy_files(train_files, train_dir)
    copy_files(validation_files, validation_dir)
    copy_files(test_files, test_dir)

print("Dataset successfully split into train, validation, and test sets.")
print(f"Total classes: {len(classes)}")

Dataset successfully split into train, validation, and test sets.
Total classes: 9


### 1.2 Split the Dataset

To ensure our model is evaluated correctly, we split the dataset into three parts:
- **Training set (70%)**: Used to train the model.
- **Validation set (15%)**: Used to tune hyperparameters and evaluate the model during training.
- **Test set (15%)**: Used to provide an unbiased evaluation of the final model.

This helps prevent data leakage and gives a more accurate measure of the model's performance on unseen data.

### 1.1 Import Libraries

This cell imports all the necessary libraries for the project, including:
- `numpy` for numerical operations.
- `tensorflow` and `keras` for building and training the deep learning models.
- `matplotlib` and `seaborn` for plotting graphs and visualizations.
- `sklearn` for performance metrics and utility functions.
- `os`, `shutil`, and `random` for file and directory operations.

In [ ]:
# Final comparison
print(f"Custom CNN Test Accuracy: {test_acc:.4f}")
print(f"MobileNetV2 Test Accuracy: {test_acc_mobilenet:.4f}")

In [ ]:
# Save the best model (MobileNetV2) to a file
model_save_path = 'boat_classifier_mobilenet.h5'
mobilenet_model.save(model_save_path)

print(f"Model saved to {model_save_path}")

### 4.3 Save the Best Model

Now that we've compared the models, we'll save the best-performing one (MobileNetV2) to a file. This will allow us to use our trained model in a separate application, like a web backend, without having to retrain it every time.

### 4.2 Final Comparison

Here, we print the final test accuracies of both models to see which one performed better on our dataset.

In [ ]:
def plot_history(history, title):
    plt.figure(figsize=(12, 5))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{title} - Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{title} - Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.show()

plot_history(history_cnn, 'Custom CNN')
plot_history(history_mobilenet, 'MobileNetV2')

## 4. Compare Models and Visualize Results

Finally, we'll compare the performance of the two models and visualize their training history.

### 4.1 Plot Training History

These plots show the model's `loss` and `accuracy` on both the training and validation sets over epochs. They are useful for diagnosing issues like overfitting (where training accuracy is much higher than validation accuracy).

In [ ]:
# Evaluate on the test set
test_loss_mobilenet, test_acc_mobilenet = mobilenet_model.evaluate(test_generator)
print(f"MobileNetV2 Test Accuracy: {test_acc_mobilenet:.4f}")

# Get predictions
y_pred_mobilenet = np.argmax(mobilenet_model.predict(test_generator), axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred_mobilenet, target_names=class_labels))

# Confusion matrix
conf_matrix_mobilenet = confusion_matrix(y_true, y_pred_mobilenet)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix_mobilenet, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('MobileNetV2 Confusion Matrix')
plt.show()

### 3.3 Evaluate the MobileNetV2 Model

We evaluate the MobileNetV2 model on the test set, just as we did for the CNN.

In [ ]:
# Train the model
history_mobilenet = mobilenet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stopping]
)

### 3.2 Train the MobileNetV2 Model

We train this model similarly to the CNN, but we don't need `class_weights` here, as transfer learning models are often more robust to imbalance.

In [ ]:
# Load the base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Build the model
mobilenet_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
mobilenet_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

mobilenet_model.summary()

## 3. Build and Train MobileNetV2 Model (Transfer Learning)

Next, we'll use a pre-trained model, **MobileNetV2**, for transfer learning. This approach leverages a model that has already been trained on a massive dataset (ImageNet) and adapts it for our specific task.

### 3.1 Load the Pre-trained Model

We load MobileNetV2 without its final classification layer (`include_top=False`) and freeze its weights (`trainable=False`) so that we only train our custom layers.

In [ ]:
# Evaluate on the test set
test_loss, test_acc = cnn_model.evaluate(test_generator)
print(f"CNN Test Accuracy: {test_acc:.4f}")

# Get predictions
y_pred = np.argmax(cnn_model.predict(test_generator), axis=1)
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('CNN Confusion Matrix')
plt.show()

### 2.3 Evaluate the CNN Model

After training, we evaluate the model's performance on the unseen test set. We'll look at:
- **Test Accuracy**: The overall percentage of correct predictions.
- **Classification Report**: A detailed breakdown of precision, recall, and F1-score for each class.
- **Confusion Matrix**: A visual representation of the model's predictions, showing where it gets confused.

In [ ]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history_cnn = cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,  # Increased epochs, but early stopping will prevent overfitting
    class_weight=class_weights_dict,
    callbacks=[early_stopping]
)

### 2.2 Train the CNN Model

We train the model using the `fit` method.
- **Early Stopping**: We use a callback to stop training if the validation loss doesn't improve for 5 consecutive epochs. This saves time and prevents overfitting.
- **Class Weights**: The `class_weights_dict` is passed to ensure balanced training.

In [ ]:
cnn_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(train_generator.num_classes, activation='softmax')  # Use num_classes from generator
])

# Compile the model
cnn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

cnn_model.summary()

## 2. Build and Train a Custom CNN Model

Now, we'll build our first model, a custom Convolutional Neural Network (CNN).

### 2.1 Define the CNN Architecture

The model consists of:
- **Convolutional layers (`Conv2D`)**: To extract features like edges and textures.
- **Batch Normalization**: To stabilize and speed up training.
- **Max Pooling layers (`MaxPooling2D`)**: To reduce the spatial dimensions and computational load.
- **Global Average Pooling**: To reduce the feature maps to a single vector per map.
- **Dense layers**: For classification, with `Dropout` to prevent overfitting.
- **Softmax activation**: In the final layer to output probabilities for each of the 9 classes.

In [ ]:
# Compute class weights to handle imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

print("Class weights:", class_weights_dict)

### 1.4 Handle Class Imbalance

Some boat categories might have more images than others. This can bias the model towards the more frequent classes. To address this, we calculate `class_weights` to give more importance to under-represented classes during training.

In [ ]:
# Define image size and batch size
img_size = (224, 224)
batch_size = 32

# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescale for validation and test sets
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

### 1.3 Data Augmentation and Loading

This section prepares the data for training. We use `ImageDataGenerator` to:
- **Rescale** the pixel values from [0, 255] to [0, 1], which is a standard practice for neural networks.
- **Augment** the training data by applying random transformations like rotation, zoom, and flips. This helps the model generalize better and reduces overfitting.
- **Load** the data from the directories we created, resizing images to a standard `(224, 224)` size.